This program creates a monthly schedule for 5 doctors.  There are two user input sections:

    1.  Requirements.  Doctors are labeled A, B, C, D, E.  
        User inputs are accepted by changing the numbers associated with the following variables:
        a. (num_days_in_month) number of days in the current month,
        b. (max_days_in_row) the maximum allowable days in a row a doctor is allowed to be scheduled,
        c. (days_unavail) days each doctor is not available.  numbers represent calendar dates,
        d. (doctor_shifts) the minimum and maximum number (inclusive) of shifts each doctor must have.
        e. (allow_day_night_double) If set to False will not allow a doctor to be scheduled for a day shift immediately
           followed by a night shift.  Takes values True and False
        f. (allow_night_day_double) If set to False will not allow a doctor to be scheduled for a night shift immediately
           followed by a day shift.  Takes values True and False
          If a schedule cannot be found to satisfy all of these requirements an error message will be given
          without producing a schedule.
        
    2.  Request preferences.  The algorithm will attempt to honor requests to the extent possible
        a.  (req_days_off) The days a doctor requests to have off.
        b.  (ideal_num_shifts) The ideal number of shifts a doctor has in the month 
        c.  (shift_type_pref) The preferred shift for each doctor
        
  (prev_month) represents the schedule for the prevous month.  Only the last 6 numbers matter in this data structure.  Its
  purpose is only to make sure the requirements are satisifed as the month crosses over.
        
        Except for the two boolean variables, only change numbers


In [18]:
#Schedule requirements.  Any created schedule must conform to these values
num_days_in_month=31
max_days_in_row=5
days_unavail = {"A": {"Day": [1,2,3], "Night": [1,2,3]}, "B": {"Day": [4,5], "Night": [4]}, "C": {"Day":[7,8,9],"Night":[7,8]}, "D": {"Day":[15],"Night":[15]}, "E": {"Day": [15], "Night": [15]}}
allow_day_night_double=False #True will allow day_night_doubles
allow_night_day_double=False #True will allow night_day_doubles
doctor_shifts = {"A": [16, 20], "B": [16, 20], "C": [16, 20], "D": [16, 20], "E": [16, 20]}
prev_month=None  

#requests/goals  adjust numbers to the right of the "=" sign for ideal_num_shifts and req_days_off.  Interchange "Day" and "Night" for
     #shift_type_pref
ideal_num_shifts = {"A": 16, "B": 16, "C": 20, "D": 19, "E": 18}
req_days_off = {"A": {"Day": [1,2,3], "Night": [1,2,3]}, "B": {"Day": [4,5,6], "Night": [4,5,6]}, "C": {"Day":[7,8,9],"Night":[7,8,9]}, "D": {"Day":[10,11,12],"Night":[10,11,12]}, "E": {"Day": [13,14,15], "Night": [13,14,15]}}
shift_type_pref={"A": "Day", "B": "Day", "C": "Day", "D": "Night", "E": "Night"}


#this is an example schedule for the previous month.  The schedule builder will take into account
#the ending days of this schedule when building the next month to ensure work rules are enforced

# prev_month={'A': {'Day1': [4, 5, 13, 18, 20, 25],
#   'Day2': [8, 9, 14, 15, 19],
#   'Night': [1, 2, 6, 21, 22, 26, 27, 28]},
#  'B': {'Day1': [14, 15, 19, 21, 22, 27, 28, 30],
#   'Day2': [3, 4, 7, 12, 13, 20, 26, 29, 31],
#   'Night': [8, 9]},
#  'C': {'Day1': [6, 17],
#   'Day2': [1, 2, 16, 21, 22, 27, 28],
#   'Night': [3, 7, 10, 11, 12, 13, 18, 23, 24, 29]},
#  'D': {'Day1': [3, 8, 9, 11, 23, 24, 29],
#   'Day2': [10],
#   'Night': [4, 5, 14, 15, 16, 17, 20, 25, 30, 31]},
#  'E': {'Day1': [1, 2, 7, 10, 12, 16, 26, 31],
#   'Day2': [5, 6, 11, 17, 18, 23, 24, 25, 30],
#   'Night': [19]}}



In [19]:
#run this cell to initialize scheduler

import doctor_scheduler as ds
scheduler_object=ds.DoctorScheduling(num_days_in_month, doctor_shifts, days_unavail, allow_day_night_double, allow_night_day_double, max_days_in_row, req_days_off, ideal_num_shifts, shift_type_pref, prev_month=None)
scheduler_object.assign_shifts()

Scheduler initialized successfully.


In [22]:
#search for schedules.  Adjust the weights to favor one category over the other.  For example if you really want to give
#people their requested days_off adjust the weight of days_off to say .7 and leave the others at .1.  Experiement for best 
#results.  The number in "mountain_climber" represents how many search iterations will be conduct.  Higher the better, but it
#takes more time to run.

weight={"pattern":.2,"days_off":.2,"num_shifts":.4,"shift_type":2}
#scheduler_object.mountain_climber(100000,weight)
scheduler_object.simulated_annealing(100000,weight)

current loss:  0.5766689075630254
final loss:  0.5441996609169983


In [23]:
#run this cell to view some schedule stats and the actual created schedule itself.

scheduler_object.calculate_percentage_days_off(scheduler_object.scheduler)
print("")
scheduler_object.actual_vs_requested_shifts(scheduler_object.scheduler)
print("")
scheduler_object.percentage_of_preferred_shift(scheduler_object.scheduler)
print("")
scheduler_object.is_schedule_legal(scheduler_object.scheduler,True,None)
print("")
scheduler_object.print_horizontal(scheduler_object.scheduler)

A  percent or requested days off actually off: 100.0
B  percent or requested days off actually off: 100.0
C  percent or requested days off actually off: 100.0
D  percent or requested days off actually off: 88.89
E  percent or requested days off actually off: 100.0

A  req: 16  actual shifts: 17
B  req: 16  actual shifts: 19
C  req: 20  actual shifts: 20
D  req: 19  actual shifts: 19
E  req: 18  actual shifts: 18

A  percent of preferred shifts to actual shifts:  0.9411764705882353
B  percent of preferred shifts to actual shifts:  0.8947368421052632
C  percent of preferred shifts to actual shifts:  0.95
D  percent of preferred shifts to actual shifts:  0.631578947368421
E  percent of preferred shifts to actual shifts:  0.8333333333333334

No day to night double shifts scheduled
No night to day double shifts scheduled
No one scheduled for more than  5  in a row
No one scheduled on their unavailable day
all shift number constraints are complied with



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A,·,·,·,D1,D1,D2,D2,D2,·,D1,D2,D2,·,·,D1,·,·,D1,D2,D2,N,·,·,·,D2,D1,D2,D1,·,·,·
B,D2,D2,D1,·,·,·,D1,N,N,·,·,·,D1,D2,D2,D1,D2,·,·,·,D1,D2,D1,D1,·,·,·,D2,D2,D1,D1
C,D1,D1,D2,D2,D2,·,·,·,·,D2,D1,·,D2,D1,N,·,·,D2,D1,D1,·,·,D2,D2,D1,D2,·,·,D1,D2,D2
D,·,N,N,N,N,N,·,D1,D1,·,·,D1,N,N,·,D2,D1,·,·,·,D2,D1,·,·,N,N,N,·,·,N,N
E,N,·,·,·,·,D1,N,·,D2,N,N,N,·,·,·,N,N,N,N,N,·,N,N,N,·,·,D1,N,N,·,·
